In [30]:
from feedgen.feed import FeedGenerator
import contents
from dateutil import parser
import markdown
import pytz


In [31]:
def convert_to_datetime(date_str):
    """
    Convert a naive or non-naive date/datetime string to a datetime object.
    Naive datetime strings are assumed to be in GMT (UTC) timezone.
    
    Args:
        date_str (str): The date or datetime string to convert.
        
    Returns:
        datetime: The corresponding datetime object.
    """
    try:
        dt = parser.parse(date_str)
        if dt.tzinfo is None:
            # If the datetime object is naive, set it to GMT (UTC)
            dt = dt.replace(tzinfo=pytz.UTC)
        return dt
    except (ValueError, TypeError) as e:
        print(f"Error parsing date string: {e}")
        return None

In [32]:
def filter_posts_by_tag(posts: list[dict], tag: str):
    """
    Generator function that filters posts by a single tag.
    """
    for post in posts:
        if tag in post.get('tags', []):
            yield post    

In [33]:
def github_markdown_to_html(markdown_str):
    """
    Convert a GitHub-flavored Markdown string to HTML.
    
    Args:
        markdown_str (str): The GitHub-flavored Markdown string to convert.
        
    Returns:
        str: The resulting HTML string.
    """
    html = markdown.markdown(
        markdown_str,
        extensions=['extra', 'codehilite', 'toc', 'tables']
    )
    return html

In [34]:
posts = contents.list_posts()
posts

[{'date': '2024-06-24T14:28:10.494486',
  'description': 'On the 1st and 2nd of July is the first-ever London Tech Zero Hackathon, supported by Kraken Tech.',
  'published': True,
  'tags': ['python', 'django', 'hackathon', 'kraken'],
  'title': 'London Tech Zero Hackathon on July 1 and 2!',
  'image': '/images/tech-zero-hackathon-square.png',
  'twitter_image': '/images/tech-zero-hackathon-square.png',
  'og_url': 'https://daniel.feldroy.com/posts/2024-06-london-hackathon-on-july-1-and-2',
  'slug': '2024-06-london-hackathon-on-july-1-and-2'},
 {'date': '2024-06-07T11:13:05.553336',
  'description': 'Mypy needs an extra identifier to not choke on an exception passed as an argument.',
  'image': '/logos/til-1.png',
  'published': True,
  'tags': ['TIL'],
  'title': 'TIL: Passing exceptions as arguments in Python',
  'twitter_image': '/logos/til-1.png',
  'slug': 'til-2024-06-passing-exceptions-as-arguments-in-python'},
 {'date': '2024-06-04T09:55:47.055467',
  'description': "Sometimes

In [35]:
fg = FeedGenerator()
fg.id('https://daniel.feldroy.com')
fg.title('Daniel Roy Greenfeld')
fg.author({'name': 'Daniel Roy Greenfeld', 'email': 'daniel@feldroy.com', 'uri':'https://daniel.feldroy.com'})
fg.link(href='https://daniel.feldroy.com', rel='alternate')
fg.title('Inside the head of Daniel Roy Greenfeld')
fg.logo('https://daniel.feldroy.com/images/pydanny-cartwheel.png')
fg.rights('All rights reserved 2024, Daniel Roy Greenfeld')
fg.language('en')

'en'

In [36]:
def add_entry(fg, raw):
    content, metadata = contents.get_post(raw['slug'])
    fe = fg.add_entry()
    uri = f'https://daniel.feldroy.com/posts/{raw["slug"]}'
    fe.id(raw['slug'])
    fe.link(href=raw['slug'])
    fe.title(metadata['title'])
    fe.summary(metadata['description'] or '')    
    fe.content(content=github_markdown_to_html(content), type='CDATA')
    # fe.category(metadata['tags'])    
    fe.contributor([{'name': 'Daniel Roy Greenfeld', 'email': 'daniel@feldroy.com'}])
    fe.author([{'name': 'Daniel Roy Greenfeld', 'email': 'daniel@feldroy.com'}])
    fe.pubDate(convert_to_datetime(metadata['date']))
    # Add tags to the entry
    for tag in metadata.get('tags', []):
        fe.category(term=tag)    

In [37]:
posts[0]
for raw in posts[:3]:
    add_entry(fg, raw)

# for index, raw in enumerate(filter_posts_by_tag(posts, 'django')):
#     add_entry(fg, raw)    
#     if index == 2: break


# print(fg.atom_str(pretty=True).decode())
fg.atom_file('feeds/atom.xml', pretty=True)

In [38]:
# from rich import print
# print(fg.atom_str(pretty=True).decode())